Obsolete: a detruire ?

In [13]:
import os

# Désactiver le GPU en définissant CUDA_VISIBLE_DEVICES à un vide#
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.tokenizeString
importlib.reload(RktnChallenge.preprocessing.tokenizeString)
from RktnChallenge.preprocessing.tokenizeString import tokenizeString

import RktnChallenge.preprocessing.filterStopWords
importlib.reload(RktnChallenge.preprocessing.filterStopWords)
from RktnChallenge.preprocessing.filterStopWords import filterStopWords


import RktnChallenge.preprocessing.mergeFeatures
importlib.reload(RktnChallenge.preprocessing.mergeFeatures)
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures

import RktnChallenge.preprocessing.mostOccur
importlib.reload(RktnChallenge.preprocessing.mostOccur)
from RktnChallenge.preprocessing.mostOccur import mostOccur

import RktnChallenge.preprocessing.Dropper
importlib.reload(RktnChallenge.preprocessing.Dropper)
from RktnChallenge.preprocessing.Dropper import Dropper

import RktnChallenge.preprocessing.TokenListToString
importlib.reload(RktnChallenge.preprocessing.TokenListToString)
from RktnChallenge.preprocessing.TokenListToString import TokenListToString

2023-09-08 00:39:32.054106: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 00:39:32.142707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 00:39:32.917427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np



from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW

# Defining constants

MAX_LEN = 128
batch_size = 16
epochs = 10
num_labels = 27
learning_rate = 0.001

RktnModel = ModelTrainer("data_tr.csv")

/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preprocess Camembert - calcul des embeddings

In [5]:
import tensorflow as tf
from transformers import CamembertTokenizer, TFCamembertForSequenceClassification, CamembertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

texts = RktnModel.data.tr_designation.values.tolist()
display(texts)
labels = RktnModel.encodeLabel("prdtypecode")

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

print("Tokenizing")
tokenizer = CamembertTokenizer.from_pretrained("camembert-base", do_lower_case=True)




# Tokenize les données d'entraînement
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length, return_tensors="tf")
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(batch_size)

# Tokenize les données de test
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length, return_tensors="tf")
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(batch_size)



config = CamembertConfig.from_pretrained("camembert-base", num_labels=num_labels)
model = TFCamembertForSequenceClassification.from_pretrained("camembert-base", config=config)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print("compile")
model.compile(optimizer=optimizer, loss=loss,   metrics=['accuracy'])


print("fit")
model.fit(train_dataset, epochs=epochs)

y_pred = model.predict(test_dataset)
y_pred_classes = tf.argmax(y_pred['logits'], axis=1).numpy()
weighted_f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Weighted F1 Score:", weighted_f1)

['Olivia: Personalisiertes Notizbuch / 150 Seiten / Punktraster / Ca Din A5 / Rosen-Design',
 "Journal Des Arts (Le) Ndeg 133 Du 28/09/2001 - L'art Et Son Marche Salon D'art Asiatique A Paris - Jacques Barrere - Francois Perrier - La Reforme Des Ventes Aux Encheres Publiques - Le Sna Fete Ses Cent Ans.",
 'Grand Stylet Ergonomique Bleu Gamepad Nintendo Wii U - Speedlink Pilot Style',
 'Peluche Donald - Europe - Disneyland 2000 (Marionnette A Doigt)',
 'La Guerre Des Tuques',
 'Afrique Contemporaine Ndeg 212 Hiver 2004 - Dossier Japon / Afrique',
 'Christof E: Sur la piste des processus éducatifs',
 'Conquerant Sept Cahier Couverture Polypro 240 X 320 Mm 96 Pages 90g Seyes Incolore',
 'Puzzle Scooby-Doo Avec Poster 2x35 Pieces',
 'Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - Longueur : 4m50 Largeur : 3 M Blanc H',
 'Eames Inspired Sxw Chair - Pink - Black',
 'Fauteuil Chesterfield Brenton 100% Cuir De Buffle - Vert Empire',
 'Peaceable Kingdom Wheres Bear? The Hide And Find Stacking Bl

Tokenizing


KeyboardInterrupt: 